#**Step1: Install amd import libraries**

In [ ]:
!pip install -q "transformers==4.35" "peft==0.5.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"
import torch
import transformers
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
import time
print('transformers.__version__: ',transformers.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.4.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf

#**Step 2: Data Loading**

In [ ]:
# Dataset
train_dataset = load_dataset("eli5_category",split="train")
validation_dataset = load_dataset("eli5_category",split="validation1[:500]")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

## 2-1 Login to WandB

In [ ]:
!pip install -q wandb
import wandb
key_api = '541b559339e102d32d9c98eb3ad2918f7703c4b1'
wandb.login(key=key_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## 2-1 Prepare dataset to train

In [ ]:

# Create a dictionary
def organize_dataset(data):
    questions = []
    answers_list = []
    scores_list = []

    for item in data:
        answers = item['answers']['text']
        scores = item['answers'].get('score', [])

        questions.append(item['title'])
        answers_list.append(answers)
        scores_list.append(scores)
    return {
        "questions": questions,
        "answers": answers_list,
        "scores": scores_list
    }



train_data = organize_dataset(train_dataset)
train_data = {key: value for key, value in train_data.items()}
val_data = organize_dataset(validation_dataset)
val_data = {key: value for key, value in val_data.items()}
print('Now we have train_data and val_data as dictionareis:')

## 2-2 Join all answers together

In [ ]:
concatenate_train_answer = []
for answe_list in train_data['answers']:
    concatenate_train_answer.append(' '.join(answe_list))

concatenate_val_answer = []
for answe_list in val_data['answers']:
    concatenate_val_answer.append(' '.join(answe_list))


print(len(concatenate_train_answer))
print(len(concatenate_val_answer))


25000
500


In [ ]:
train_data['answers'] = concatenate_train_answer
train_data['answers'][0]

'I work in film and have a VFX degree and here\'s how it goes: 1. About half the money, give or take, is for above the line talent. So you have your actors, directors, producers, ect. They get paid in a percentage or in absurdly high amounts for films. These people are also accommodate on set so production has to rent out luxury campers to house them for weeks or months at a time when on location. Then they need to hire drivers and trucks to move those campers. Top tier stars can make demands on top of that. I saw Jim Carrey\'s camper once and it had an entire astroturf lawn on top of it, with a picnic table, with a vase with flowers on it. Don\'t ask me why he wanted it, he just did. Those costs are in addition to percentages given to the talent directly, which can be millions each for an A list celebrity. If this is a movie like Infinity War you have multiple guys like RDJ and Cumberbach and like four guy named Chris who could carry a blockbuster on their own and want to be paid like

In [ ]:
val_data['answers'] = concatenate_val_answer
val_data['answers'][0]

"Whilst I'm no expert I listened to a very good segment on LBC recently (uk talkback radio) that explained that the men are more likely to commit such abuse but the percentages of women abusers are skewed lower due to several cultural issues such as: 1. Women are automatically perceived as peaceful nurturers where as men are aggressive physicals. 2. Children are less likely to call abuse against female family members as they are less likely to be believed. 3. Female abuse is (wrongly) perceived in many cases as being invited especially where the victim is slightly older i.e. Boys must like it right. This is the one cultural aspect I hate the most because a male teacher/person in trust (for example) is automatically in the wrong and a female teacher is more likely to go free / have no charges / have no abuse claim made. Don't get me wrong - male abusers are still the largest statistic but to deny there is an issue with female abuse harms the ability of our children to come forward. It i

## 2-3 convert dict to dataframe

In [ ]:
train_data_df = pd.DataFrame(train_data)
val_data_df  = pd.DataFrame(val_data)
print('Training data')
display(train_data_df.head(5))
print('Validataion:')
display(val_data_df.head(5))

Training data


,questions,answers,scores
0,Why do Marvel movies (and other heavily CGI- a...,I work in film and have a VFX degree and here'...,"[10751, 8991, 1222, 297, 153, 124, 106, 58, 36..."
1,"when elephants suck water with their trunks, h...",Imagine having a straw in glass of water. Now ...,"[7, 4]"
2,Why is it that I sometimes get just one single...,"They’re not taste buds, they’re lingual papill...","[32, 21]"
3,What exactly is a coma and how does our body d...,A coma is a state of unconsciousness which som...,"[789, 16, 7, 4]"
4,Why is it harder to learn things as an adult?,it's not harder. it's just that by the time yo...,[4]


Validataion:


,questions,answers,scores
0,why is paedophilia so much more common in men ...,Whilst I'm no expert I listened to a very good...,"[22, 12, 6, 5, 4]"
1,Why is it okay to make fun of people who are f...,Honestly I don't know but as a southerner I fi...,[5]
2,"Why do we, as humans, crave social interaction...",Lots of people prefer to do things on their ow...,[3]
3,"What was Nietzche's philosophy, exactly?","Nietzsche's ""Ubermensch"" is the goal that soci...","[686, 172, 69, 35, 26, 10, 7, 7, 6, 3, 3]"
4,The Political Spectrum,The political spectrum varies widely from coun...,[11]


## 2-4 Use a structure for model input

In [ ]:
# Concatenate 'Question' and 'Answer' columns into a new column 'text'
train_data_df['text'] = 'Question:\n' + train_data_df['questions'] + '\n\nAnswer:\n' + train_data_df['answers']
val_data_df['text'] = 'Question:\n' + val_data_df['questions'] + '\n\nAnswer:\n' + val_data_df['answers']
train_data_df.drop(labels=['questions','answers','scores'],axis=1,inplace=True)
val_data_df.drop(labels=['questions','answers','scores'],axis=1,inplace=True)

In [ ]:
display(train_data_df.head(5))
display(val_data_df.head(5))

,text
0,Question:\nWhy do Marvel movies (and other hea...
1,Question:\nwhen elephants suck water with thei...
2,Question:\nWhy is it that I sometimes get just...
3,Question:\nWhat exactly is a coma and how does...
4,Question:\nWhy is it harder to learn things as...


,text
0,Question:\nwhy is paedophilia so much more com...
1,Question:\nWhy is it okay to make fun of peopl...
2,"Question:\nWhy do we, as humans, crave social ..."
3,"Question:\nWhat was Nietzche's philosophy, exa..."
4,Question:\nThe Political Spectrum\n\nAnswer:\n...


In [ ]:
print(train_data_df['text'].loc[0])

Question:
Why do Marvel movies (and other heavily CGI- and animation-based films) cost so much to produce? Where do the hundreds of millions of dollars go to, exactly?

Answer:
I work in film and have a VFX degree and here's how it goes: 1. About half the money, give or take, is for above the line talent. So you have your actors, directors, producers, ect. They get paid in a percentage or in absurdly high amounts for films. These people are also accommodate on set so production has to rent out luxury campers to house them for weeks or months at a time when on location. Then they need to hire drivers and trucks to move those campers. Top tier stars can make demands on top of that. I saw Jim Carrey's camper once and it had an entire astroturf lawn on top of it, with a picnic table, with a vase with flowers on it. Don't ask me why he wanted it, he just did. Those costs are in addition to percentages given to the talent directly, which can be millions each for an A list celebrity. If this 

## 2-5 Turn dataframe to Hugging format

In [ ]:
#convert to Huggingface Datasets format
train = Dataset.from_pandas(train_data_df)
validation = Dataset.from_pandas(val_data_df)
print('Train')
display(train)
print('Validation')
display(validation)

Train


Dataset({
    features: ['text'],
    num_rows: 25000
})

Validation


Dataset({
    features: ['text'],
    num_rows: 500
})

# **Step 3: Load  base model and configuration**

In [ ]:
model_id = "NousResearch/Llama-2-7b-chat-hf"

## 3-1 Set BitsAndBytesConfig
Quantization techniques aim to represent model weights and activations using lower-precision data types (e.g., 4-bit integers) instead of full precision (e.g., 16-bit floats).
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content=", initial-scale=1.0">
    <title></title>
</head>
<body>
    <img src="https://drive.google.com/uc?export=view&id=1Y3ymMpJ4yWHUTs0fAYYJGFlQs_1bozvy" width="30%">
</body>
</html>


In [ ]:
# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)


## 3-2 Set tokenizer

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content=", initial-scale=1.0">
    <title></title>
</head>
<body>
    <img src="https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/_images/bert-tokenizer.jpeg" width="60%">
</body>
</html>



In [ ]:
%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # add padding to right ann addd eos token at the end
tokenizer.padding_side = "right"

CPU times: user 70.3 ms, sys: 1 ms, total: 71.3 ms
Wall time: 103 ms


## 3-3 Call model

In [ ]:
%%time
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto")

# Check the device of the model
def check_model_device(model):
    for param in model.parameters():
        return param.device

device_of_model = check_model_device(model)
print(f"Model is on device: {device_of_model}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 33.6 s, sys: 25.4 s, total: 58.9 s
Wall time: 1min 56s


## 3-4 Set Lora config
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title></title>
</head>
<body>
    <img src="https://drive.google.com/uc?export=view&id=1cdDI0LPOOBSEFdMq1zMSDR6_TcbpY7PY" width="75%">
</body>
</html>



**Components of LoRA:**

1. **Decomposition of Weight Matrices**:
   - LoRA begins by decomposing the weight matrices of pre-trained language models into smaller submatrices using techniques like singular value decomposition (SVD) or low-rank factorization. This process reduces the rank of the matrices while preserving their essential information.

2. **Efficient Training with Reduced Parameters**:
   - With the decomposed matrices, the model now has fewer parameters, which leads to reduced computational complexity during training. This enables more efficient fine-tuning on hardware with limited memory or computational capabilities.

3. **Retaining Model Performance**:
   - Despite the reduction in parameters, LoRA aims to retain the model's performance by carefully selecting the decomposition approach and optimizing the remaining parameters. By striking a balance between model complexity and efficiency, LoRA facilitates the deployment of powerful language models in real-world applications.






In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
                          lora_alpha=16,
                          lora_dropout=0.3,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM"
                        )

## 3-5 Set Training args

In [ ]:

args = TrainingArguments(
    output_dir='fined_tuened_lamma_model',
    num_train_epochs=1, # adjust based on the data size
    per_device_train_batch_size=4, # use 4 if you have more GPU RAM
    per_device_eval_batch_size=4,
    save_strategy="steps", #steps
    save_steps=300,
    evaluation_strategy="steps",
    logging_steps=300,
    eval_steps = 300,
    learning_rate=2e-4,
    fp16=True,
    seed=42,
    save_total_limit=3,
    load_best_model_at_end =True,
    #group_by_length= True
)

## 3-6 Set SFTTTrainer

In [ ]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=validation,
    dataset_text_field='text',
    peft_config=peft_config,

    max_seq_length=1042,
    tokenizer=tokenizer,
    args=args,
    packing=True,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


# **Step 4: Train model**

In [ ]:
# train
start = time.time()
resutl = trainer.train()
stop = time.time()
duration = (stop-start)/60
print('Duration(ms): ',duration)

wandb: Currently logged in as: amirifard-mo (mohammadamirifard). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240524_084329-fnla0upo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run twilight-puddle-16
wandb: ⭐️ View project at https://wandb.ai/mohammadamirifard/huggingface
wandb: 🚀 View run at https://wandb.ai/mohammadamirifard/huggingface/runs/fnla0upo
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.uti

Step,Training Loss,Validation Loss
300,2.293900,2.339222
600,2.251600,2.329179
900,2.245000,2.325004
1200,2.234300,2.321757
1500,2.229100,2.319216
1800,2.223400,2.318017
2100,2.233000,2.315719


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Duration(ms):  541.8986797650655


# **Step 5: Show Results**

Please see result from here. Due to hardaware restictions, we trained our model several times using updatated weights from previous step.

**Training on 15000 data** : [Link](https://api.wandb.ai/links/mohammadamirifard/65hvevrt)

**Training from 15000 to 25000 data** : [Link](https://api.wandb.ai/links/mohammadamirifard/01oczpqs)

**Training from 25000 to 50000 data** : [Link](https://api.wandb.ai/links/mohammadamirifard/o7xaqrt7)

**Training from 50000 to end data** : [Link](https://api.wandb.ai/links/mohammadamirifard/vclhe8xe)




# **Step 6: Save model**

In [ ]:
# save model in local
trainer.save_model()